In [178]:
import pandas as pd
import numpy as np

In [179]:
applicants = pd.read_csv(r"C:\Users\maghanga\Desktop\datasets\application_record.csv")
records = pd.read_csv(r"C:\Users\maghanga\Desktop\datasets\credit_record.csv")

In [180]:
applicants.head()

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [181]:
print(applicants.columns)
print(records.columns)

Index(['ID', 'CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY', 'CNT_CHILDREN',
       'AMT_INCOME_TOTAL', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE',
       'NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE', 'DAYS_BIRTH',
       'DAYS_EMPLOYED', 'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE',
       'FLAG_EMAIL', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS'],
      dtype='object')
Index(['ID', 'MONTHS_BALANCE', 'STATUS'], dtype='object')


In [182]:
applicants.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car Owner','FLAG_OWN_REALTY':'Property Owner',
                         'CNT_CHILDREN':'ChldNo','AMT_INCOME_TOTAL':'annual_income',
                         'NAME_EDUCATION_TYPE':'Education','NAME_FAMILY_STATUS':'Marital Status',
                        'NAME_HOUSING_TYPE':'Housing','FLAG_EMAIL':'email',
                         'NAME_INCOME_TYPE':'Income Source','FLAG_WORK_PHONE':'wkphone',
                         'FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'Family Size',
                        'OCCUPATION_TYPE':'Occupation'
                        },inplace=True)

In [183]:
applicants['DAYS_BIRTH'] = [abs(record/365) for record in applicants['DAYS_BIRTH']]
applicants['DAYS_EMPLOYED'] = [abs(record/365) for record in applicants['DAYS_EMPLOYED']]
applicants.rename(columns={'DAYS_BIRTH': 'Age', 'DAYS_EMPLOYED': 'YEARS_WORKED'}, inplace=True)

In [184]:
merged_data = pd.merge(applicants, records, how='inner', on='ID')
merged_data.shape

(777715, 20)

In [185]:
merged_data.columns

Index(['ID', 'Gender', 'Car Owner', 'Property Owner', 'ChldNo',
       'annual_income', 'Income Source', 'Education', 'Marital Status',
       'Housing', 'Age', 'YEARS_WORKED', 'FLAG_MOBIL', 'wkphone', 'phone',
       'email', 'Occupation', 'Family Size', 'MONTHS_BALANCE', 'STATUS'],
      dtype='object')

In [186]:
merged_data.head()

,ID,Gender,Car Owner,Property Owner,ChldNo,annual_income,Income Source,Education,Marital Status,Housing,Age,YEARS_WORKED,FLAG_MOBIL,wkphone,phone,email,Occupation,Family Size,MONTHS_BALANCE,STATUS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,0,C
1,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,-1,C
2,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,-2,C
3,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,-3,C
4,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,32.890411,12.443836,1,1,0,0,NaN,2.0,-4,C


In [187]:
merged_data.dropna(inplace=True)


In [188]:
# encoding data
merged_data['Gender'] = merged_data['Gender'].replace(['F','M'],[0,1])
merged_data['Car Owner'] = merged_data['Car Owner'].replace(['N','Y'],[0,1])
merged_data['Property Owner'] = merged_data['Property Owner'].replace(['N','Y'],[0,1])


Creating the response variable

In [189]:
# creating a new column--> the response variable
merged_data['Good Client'] = ['YES' if record == 'C' else 'NO'  for record in merged_data['STATUS']]

In [190]:
good_clients = merged_data.loc[merged_data['Good Client']=='YES']
print("Good clients shape:", good_clients.shape)
bad_clients = merged_data.loc[merged_data['Good Client']=='NO']
print('Bad clients shape:', bad_clients.shape)

Good clients shape: (226185, 21)
Bad clients shape: (311482, 21)


In [191]:
merged_data.to_csv(r'C:\Users\maghanga\Desktop\datasets\merged_data.csv')

In [192]:
clients_who_might_be_good = merged_data[(merged_data['STATUS'] == '0') | (merged_data['STATUS'] == 'X')].index

merged_data.drop(clients_who_might_be_good, inplace=True)

In [193]:
# dropping clients who have been classified as both bad and good
merged_data.drop_duplicates(subset=['ID', 'Good Client'], keep=False, inplace=True)

In [194]:
# feature reduction
merged_data.drop(columns=['ID', 'FLAG_MOBIL', 'STATUS', 'MONTHS_BALANCE', 'email', 'wkphone', 'phone'], inplace=True)

In [195]:
merged_data.head(2)

,Gender,Car Owner,Property Owner,ChldNo,annual_income,Income Source,Education,Marital Status,Housing,Age,YEARS_WORKED,Occupation,Family Size,Good Client
301,0,1,0,0,130500.0,Working,Incomplete higher,Married,House / apartment,29.230137,3.021918,Accountants,2.0,NO
873,0,1,1,2,135000.0,Working,Secondary / secondary special,Married,House / apartment,43.180822,8.693151,Laborers,4.0,YES


In [196]:
def format_string(string_arg):
    new_list = string_arg.split('/')
    if new_list[0] == 'House':
        return new_list[1]
    return new_list[0]

In [197]:
# removing the forward-slash
merged_data['Education'] = [format_string(record) for record in merged_data['Education']]
merged_data['Housing'] = [format_string(record) for record in merged_data['Housing']]
merged_data['Marital Status'] = [format_string(record) for record in merged_data['Marital Status']]

In [198]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

merged_data['Education'] = encoder.fit_transform(merged_data['Education'])
merged_data['Housing'] = encoder.fit_transform(merged_data['Housing'])
merged_data['Marital Status'] = encoder.fit_transform(merged_data['Marital Status'])
merged_data['Occupation'] = encoder.fit_transform(merged_data['Occupation'])
merged_data['Income Source'] = encoder.fit_transform(merged_data['Income Source'])

In [199]:
merged_data.head()

,Gender,Car Owner,Property Owner,ChldNo,annual_income,Income Source,Education,Marital Status,Housing,Age,YEARS_WORKED,Occupation,Family Size,Good Client
301,0,1,0,0,130500.0,3,2,1,1,29.230137,3.021918,0,2.0,NO
873,0,1,1,2,135000.0,3,4,1,1,43.180822,8.693151,8,4.0,YES
962,1,1,1,0,360000.0,0,4,1,1,45.671233,14.695890,16,2.0,NO
1544,0,0,1,0,297000.0,0,4,3,4,42.517808,8.860274,8,1.0,NO
1581,0,0,1,0,297000.0,0,4,3,4,42.517808,8.860274,8,1.0,NO


Splitting feature and response variables

In [200]:
X = merged_data.drop(columns=['Good Client']).values
y = merged_data['Good Client'].values

In [201]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [202]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X, y)



DecisionTreeClassifier()

In [203]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

predictions = model.predict(X_test)

print(classification_report(y_test, predictions))
print("----------------------------------------")
print(confusion_matrix(y_test, predictions))
print("----------------------------------------")
print("Score:", accuracy_score(y_test, predictions))

              precision    recall  f1-score   support

          NO       0.95      1.00      0.97       307
         YES       0.98      0.76      0.86        67

    accuracy                           0.95       374
   macro avg       0.97      0.88      0.92       374
weighted avg       0.96      0.95      0.95       374

----------------------------------------
[[306   1]
 [ 16  51]]
----------------------------------------
Score: 0.9545454545454546
